In [342]:
import numpy as np

In [343]:
class Dense:
    def __init__(self, input_dim, output_dim):
        self.weights = 0.01* np.random.randn(input_dim, output_dim)
        self.biases = np.zeros((1,output_dim))
    
    def forward(self, x):
        """X = [[1,2,3] x1 rows, 
            [5,6,9] x2 rows,    
            [11,6,9]] x3 rows 
         weights = [[11,42,53] input = 3 , neurons = 3 , 
            [55,36,92] ,    
            [131,46,92]]
         bias = [1,4,7]
         result = [x1* weights[0] + bias[0], x2* weights[1] + bias[1]],x2* weights[2] + bias[2]
         """
        self.input = x           # actually x = [[1,3],....[1,5] 300(cuz of 3 classes),2] 
        self.output = np.dot(x, self.weights) + self.biases # actually x * weights=[[]]
        
    
    def backward(self, dL_dout):
        self.dweights = dL_dout.T @ self.input  
        self.dbias = np.sum(dL_dout, axis=0) 
        dL_dinput = dL_dout @ self.weights  
        return dL_dinput


In [344]:
class ReluAct:
    def forward(self, x):
        self.input = x
        self.output = np.maximum(0, x)
        
    
    def backward(self, dL_dout):
        dL_dinput = dL_dout * (self.input > 0).astype(float)
        return dL_dinput


In [345]:
class Softmax:
    def forward(self, x):
        max_row = np.max(x, axis=1, keepdims=True)
        normalize_row = x - max_row
        expo = np.exp(normalize_row)
        self.output = expo / np.sum(expo, axis=1, keepdims=True)
    



In [346]:
class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        if y_true.ndim == 2 and y_true.shape[1] != y_pred.shape[1]:
            raise ValueError("y_true shape does not match y_pred classes")
        if y_true.ndim == 1 or (y_true.ndim == 2 and y_true.shape[1] == 1):
            y_one_hot = np.zeros_like(y_pred)
            y_one_hot[np.arange(y_true.size), y_true.flatten()] = 1
            y_true = y_one_hot
        
        y_pred_clipped = np.clip(y_pred, 1e-12, 1.0)
        sample_losses = -np.sum(y_true * np.log(y_pred_clipped), axis=1)
        self.y_pred = y_pred
        self.y_true = y_true
        self.output = np.mean(sample_losses)
    
    def backward(self):
        n_samples = self.y_true.shape[0]
        dL_dy = (self.y_pred - self.y_true) / n_samples
        return dL_dy



In [347]:
    
class SGDOptimizer:
    def __init__(self,betaDecay,learning_rate=0.001):
        self.lr = learning_rate
        self.beta = betaDecay
        self.momentum = None  
    def update(self,layer):
        if self.beta==0:
            layer.weights = layer.weights - self.lr * layer.dweights 
            layer.biases = layer.biases - self.lr * layer.dbias
        else:
            if self.momentum is None:
                self.momentum = SGDOptimizerMomentum(layer, self.lr, self.beta)
            self.momentum.update()
        
    
        


In [348]:
np.random.seed(42)

In [349]:
from nnfs.datasets import spiral_data
X, y = spiral_data(samples=100, classes=3)

dense1 = Dense(2, 64)
activation1 = ReluAct()
dense2 = Dense(64, 3)
softmax = Softmax()
loss_func = CrossEntropyLoss()
optimizer = SGDOptimizer(learning_rate=0.01,betaDecay=0)



In [350]:
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
softmax.forward(dense2.output)
loss_func.forward(softmax.output,y)
predictions = np.argmax(loss_func.output)
if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

print(accuracy)

0.3333333333333333


In [351]:
loss_func.output

np.float64(1.0985852231839113)